In [1]:
import os
import numpy as np
import pandas as pd
from utility_functions import get_acc, add_row_to_csv

In [2]:
folder_path = 'training_data'
datasets = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]

In [3]:
category = 'proposed'
model = "linear_piecewise_2_regions"
n_features_list = [4]
# datasets = ['H3K4me3_PGP_immune']

In [4]:
prediction_folder_path = 'model/' + model + '/predictions/'

In [5]:
def get_acc_from(dataset, test_fold, n_feature):
    evaluation_df = pd.read_csv('training_data/' + dataset + '/evaluation.csv')
    fold_df = pd.read_csv('training_data/' + dataset + '/folds.csv')
    eval_df = evaluation_df[evaluation_df['sequenceID'].isin(fold_df[fold_df['fold'] == test_fold]['sequenceID'])]
    lldas_df = pd.read_csv(prediction_folder_path + category + "." + dataset + '.' + str(test_fold) + '.' + str(n_feature) + '.csv')
    return get_acc(eval_df, lldas_df)

In [6]:
def extract_between_dots(name):
        parts = name.rsplit('.')
        return parts[-2]

In [7]:
for dataset in datasets:
    # get number of folds
    fold_df = pd.read_csv('training_data/' + dataset + '/folds.csv')
    n_folds = np.unique(fold_df['fold']).__len__()

    for test_fold in range(1, n_folds + 1):
        for n_feature in n_features_list:
            try:
                acc = get_acc_from(dataset, test_fold, n_feature)
                # print(dataset, test_fold, n_feature, acc)
                add_row_to_csv('acc_rate_csvs/' + dataset + '.csv', 
                               ['method', 'fold', 'acc'],
                               [model + '.' + str(n_feature), test_fold, acc])
            except Exception as e:
                print(f"Error calculating accuracy for dataset {dataset}, fold {test_fold}, n_feature {n_feature}: {e}")